In [1]:
from netort.data_manager import DataSession
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

def get_uploader(data_session, column_mapping, hostname, source):
    """

    :type data_session: DataSession
    """
    # router = {col_name: data_session.new_tank_metric('metrics', metric_name,
    #                                                  hostname, group, source)
    #           for col_name, (metric_name, group) in column_mapping}
    _router = {}
    def get_router(tags):
        if set(tags) - set(_router.keys()):
            [_router.setdefault(tag,
                               {col_name:
                                    data_session.new_aggregated_metric(name,
                                                                 # hostname,
                                                                 group,
                                                                 # source,
                                                                 ammo_tag=tag)
                                for col_name, (name, group) in column_mapping.items()})
             for tag in tags]
        return _router
    
    def upload_df(df):
        router = get_router(df.tag.unique().tolist())
        for tag, df_tag in df.groupby('tag'):
            for col_name, metric in router[tag].items():
                df_tag['value'] = df_tag[col_name]
                metric.put(df_tag)
                
    return upload_df

In [2]:
from netort.data_manager import DataSession
clients = [{'type': 'luna', 'api_address': 'https://test-back.luna.yandex-team.ru/'}]
data_session = DataSession({'clients': clients})
col_map = {name: (name, 'fractions') for name in ['connect_time', 'send_time',
           'latency', 'receive_time',
           'interval_event']}
uploader = get_uploader(data_session, col_map, 'localhost', 'tank')

INFO:Created new local data session: job_87feee97-2aac-414c-b51f-24ef482710cc
DEBUG:Data from routing queue:
[]
DEBUG:No subscribers/callbacks for metrics yet... skipped metrics
DEBUG:Luna client local id: client_c9d425bf-075f-442d-a9ea-033e898c8914
DEBUG:Metrics for subscriber subscriber_cc424f33-7dac-4eef-b6c8-647128360d0c not found
DEBUG:DataSession clients: [<netort.data_manager.clients.luna.LunaClient object at 0x108151b50>]
DEBUG:DataSession subscribers:                                                                                           callback  \
subscriber_cc424f33-7dac-4eef-b6c8-647128360d0c  <bound method LunaClient.put of <netort.data_m...   

                                                    type  
subscriber_cc424f33-7dac-4eef-b6c8-647128360d0c  __ANY__  


In [3]:
from yandextank.plugins.Phantom.reader import string_to_df_microsec
with open('/Users/fomars/dev/netort/tmp/phout_tagged.log') as f:
    chunk = f.read()
df = string_to_df_microsec(chunk)

Traceback (most recent call last):
  File "/Users/fomars/dev/netort/netort/netort/resource.py", line 16, in <module>
    import boto
ImportError: No module named boto


In [4]:
uploader(df)

DEBUG:Found subscribers for this metric, subscribing...:                                                                                           callback  \
subscriber_cc424f33-7dac-4eef-b6c8-647128360d0c  <bound method LunaClient.put of <netort.data_m...   

                                                    type  
subscriber_cc424f33-7dac-4eef-b6c8-647128360d0c  __ANY__  
DEBUG:Found subscribers for this metric, subscribing...:                                                                                           callback  \
subscriber_cc424f33-7dac-4eef-b6c8-647128360d0c  <bound method LunaClient.put of <netort.data_m...   

                                                    type  
subscriber_cc424f33-7dac-4eef-b6c8-647128360d0c  __ANY__  
DEBUG:Found subscribers for this metric, subscribing...:                                                                                           callback  \
subscriber_cc424f33-7dac-4eef-b6c8-647128360d0c  <bound method LunaClient.put of 

DEBUG:Request https://volta-back-testing.common-int.yandex-team.ru//create_metric/ code 200. Text: 0cb43d4caa964b3bb599b112aa04249f
DEBUG:Successfully received tag 0cb43d4caa964b3bb599b112aa04249f for metric.local_id: metric_17faaa3b-d4a9-4f6b-88b8-45ed204d563d
DEBUG:Prepared create_job request:
-----------QUERY START-----------
POST https://volta-back-testing.common-int.yandex-team.ru//create_metric/
Content-Length: 147
Content-Type: application/x-www-form-urlencoded

job=813&local_id=metric_bfa7186b-eccd-48a0-bc96-9f1a69f89b3b&group=fractions&name=send_time&ammo_tag=%2Fmtinfo%2Ftrajectories%7Call&type=aggregates
-----------QUERY END-----------
DEBUG:https://volta-back-testing.common-int.yandex-team.ru:443 "POST //create_metric/ HTTP/1.1" 200 32
DEBUG:Request https://volta-back-testing.common-int.yandex-team.ru//create_metric/ code 200. Text: 98ec66a0ed2144ccaf7666d4ba4033ff
DEBUG:Successfully received tag 98ec66a0ed2144ccaf7666d4ba4033ff for metric.local_id: metric_bfa7186b-eccd-48a0

DEBUG:callbacks:
                                                                                      callback  \
metric_b9236bcc-fb3a-4111-9ded-c95d1c34278f  <bound method LunaClient.put of <netort.data_m...   
metric_035f0f9b-41fe-4a14-8d7e-74d7957e64c2  <bound method LunaClient.put of <netort.data_m...   
metric_e9af35fa-4afc-4faf-9f65-7b1e99abba05  <bound method LunaClient.put of <netort.data_m...   
metric_17faaa3b-d4a9-4f6b-88b8-45ed204d563d  <bound method LunaClient.put of <netort.data_m...   
metric_bfa7186b-eccd-48a0-bc96-9f1a69f89b3b  <bound method LunaClient.put of <netort.data_m...   
metric_ad0ffb71-a985-4029-afb4-b48815c3781a  <bound method LunaClient.put of <netort.data_m...   
metric_d46b0f2a-4d49-4565-8109-9598a439ef75  <bound method LunaClient.put of <netort.data_m...   
metric_cbfa477d-5d3f-4b2f-9761-adc67da03c88  <bound method LunaClient.put of <netort.data_m...   
metric_ccf43f74-a5d9-47fb-a980-0ae521332c6d  <bound method LunaClient.put of <netort.data_m...   
met

DEBUG:callbacks:
                                                                                      callback  \
metric_b9236bcc-fb3a-4111-9ded-c95d1c34278f  <bound method LunaClient.put of <netort.data_m...   
metric_035f0f9b-41fe-4a14-8d7e-74d7957e64c2  <bound method LunaClient.put of <netort.data_m...   
metric_e9af35fa-4afc-4faf-9f65-7b1e99abba05  <bound method LunaClient.put of <netort.data_m...   
metric_17faaa3b-d4a9-4f6b-88b8-45ed204d563d  <bound method LunaClient.put of <netort.data_m...   
metric_bfa7186b-eccd-48a0-bc96-9f1a69f89b3b  <bound method LunaClient.put of <netort.data_m...   
metric_ad0ffb71-a985-4029-afb4-b48815c3781a  <bound method LunaClient.put of <netort.data_m...   
metric_d46b0f2a-4d49-4565-8109-9598a439ef75  <bound method LunaClient.put of <netort.data_m...   
metric_cbfa477d-5d3f-4b2f-9761-adc67da03c88  <bound method LunaClient.put of <netort.data_m...   
metric_ccf43f74-a5d9-47fb-a980-0ae521332c6d  <bound method LunaClient.put of <netort.data_m...   
met

DEBUG:Luna client processing took 1000.62 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.66 ms
DEBUG:Luna client processing took 1001.54 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 5.52 ms
DEBUG:Luna client processing took 1000.15 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.48 ms
DEBUG:Luna client processing took 1003.49 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.82 ms
DEBUG:Luna client processing took 1000.10 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.56 ms
DEBUG:Luna client processing took 1004.16 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.40 ms
DEBUG:Luna client processing took 1003.04 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.64 ms
DEBUG:Luna client processing took 1003.36 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.47 ms
DEBUG:Luna client processing took 1001.56 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 

...

DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.18 ms
DEBUG:Luna client processing took 1005.02 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.97 ms
DEBUG:Luna client processing took 1003.05 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.43 ms
DEBUG:Luna client processing took 1001.80 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.33 ms
DEBUG:Luna client processing took 1001.70 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.02 ms
DEBUG:Luna client processing took 1003.17 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.66 ms
DEBUG:Luna client processing took 1002.74 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.23 ms
DEBUG:Luna client processing took 1002.22 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.01 ms
DEBUG:Luna client processing took 1004.01 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.24 ms
DEBUG:Luna client processing took 100

DEBUG:Routing cycle took 2.04 ms
DEBUG:Luna client processing took 1001.70 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.19 ms
DEBUG:Luna client processing took 1004.73 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.64 ms
DEBUG:Luna client processing took 1002.00 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.86 ms
DEBUG:Luna client processing took 1001.88 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.05 ms
DEBUG:Luna client processing took 1000.18 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.80 ms
DEBUG:Luna client processing took 1001.54 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.19 ms
DEBUG:Luna client processing took 1002.57 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.18 ms
DEBUG:Luna client processing took 1003.16 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.10 ms
DEBUG:Luna client processing took 1002.40 ms
DEBUG:Data from routing qu

DEBUG:Luna client processing took 1002.74 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.61 ms
DEBUG:Luna client processing took 1000.91 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.62 ms
DEBUG:Luna client processing took 1004.99 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.03 ms
DEBUG:Luna client processing took 1005.18 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.61 ms
DEBUG:Luna client processing took 1002.42 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.31 ms
DEBUG:Luna client processing took 1001.38 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.83 ms
DEBUG:Luna client processing took 1002.52 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.14 ms
DEBUG:Luna client processing took 1002.29 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.62 ms
DEBUG:Luna client processing took 1002.62 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 

DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.74 ms
DEBUG:Luna client processing took 1004.67 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.61 ms
DEBUG:Luna client processing took 1002.90 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.59 ms
DEBUG:Luna client processing took 1002.17 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.86 ms
DEBUG:Luna client processing took 1001.40 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.25 ms
DEBUG:Luna client processing took 1003.61 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.50 ms
DEBUG:Luna client processing took 1000.19 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.46 ms
DEBUG:Luna client processing took 1003.27 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.66 ms
DEBUG:Luna client processing took 1002.33 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.70 ms
DEBUG:Luna client processing took 100

DEBUG:Routing cycle took 2.59 ms
DEBUG:Luna client processing took 1003.39 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 9.43 ms
DEBUG:Luna client processing took 1003.58 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.37 ms
DEBUG:Luna client processing took 1001.65 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.25 ms
DEBUG:Luna client processing took 1002.92 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.18 ms
DEBUG:Luna client processing took 1000.32 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 2.91 ms
DEBUG:Luna client processing took 1004.74 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.83 ms
DEBUG:Luna client processing took 1005.25 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 7.98 ms
DEBUG:Luna client processing took 1000.11 ms
DEBUG:Data from routing queue:
[]
DEBUG:Routing cycle took 1.89 ms
DEBUG:Luna client processing took 1001.12 ms
DEBUG:Data from routing qu

In [10]:
data_session.update_job({'name': 'aggr_job1'})

DEBUG:Prepared update_job request:
-----------QUERY START-----------
POST https://volta-back-testing.common-int.yandex-team.ru//update_job/?job=803
Content-Length: 14
Content-Type: application/x-www-form-urlencoded

name=aggr_job1
-----------QUERY END-----------
DEBUG:Routing cycle took 0.09 ms
DEBUG:Luna client processing took 1003.85 ms
DEBUG:https://volta-back-testing.common-int.yandex-team.ru:443 "POST //update_job/?job=803 HTTP/1.1" 200 0
DEBUG:Request https://volta-back-testing.common-int.yandex-team.ru//update_job/?job=803 code 200. Text: 
DEBUG:Update job status: 200
DEBUG:Answ data: 
DEBUG:Client job updated: <netort.data_manager.clients.luna.LunaClient object at 0x10ede3650>
